In [1]:
import pandas as pd
import numpy as np

#### Load Train and Test Data:

In [2]:
train = pd.read_csv(r"/Users/shabnanasser/workplace/git/Capstone_Two/Train-1542865627584.csv")
train_beneficiary = pd.read_csv(r"/Users/shabnanasser/workplace/git/Capstone_Two/Train_Beneficiarydata-1542865627584.csv")
train_inpatient = pd.read_csv(r"/Users/shabnanasser/workplace/git/Capstone_Two/Train_Inpatientdata-1542865627584.csv")
train_outpatient = pd.read_csv(r"/Users/shabnanasser/workplace/git/Capstone_Two/Train_Outpatientdata-1542865627584.csv")

In [3]:
test = pd.read_csv(r"/Users/shabnanasser/workplace/git/Capstone_Two/Test-1542969243754.csv")
test_beneficiary = pd.read_csv(r"/Users/shabnanasser/workplace/git/Capstone_Two/Test_Beneficiarydata-1542969243754.csv")
test_inpatient = pd.read_csv(r"/Users/shabnanasser/workplace/git/Capstone_Two/Test_Inpatientdata-1542969243754.csv")
test_outpatient = pd.read_csv(r"/Users/shabnanasser/workplace/git/Capstone_Two/Test_Outpatientdata-1542969243754.csv")

#### Shape of data:

In [4]:
print('Shape of train_y data :',train.shape)
print('Shape of train_beneficiary data :',train_beneficiary.shape)
print('Shape of train_inpatient data :',train_inpatient.shape)
print('Shape of train_outpatient data :',train_outpatient.shape)

print('Shape of test_provs data :',test.shape)
print('Shape of test_beneficiary data :',test_beneficiary.shape)
print('Shape of test_inpatient data :',test_inpatient.shape)
print('Shape of test_outpatient data :',test_outpatient.shape)

Shape of train_y data : (5410, 2)
Shape of train_beneficiary data : (138556, 25)
Shape of train_inpatient data : (40474, 30)
Shape of train_outpatient data : (517737, 27)
Shape of test_provs data : (1353, 1)
Shape of test_beneficiary data : (63968, 25)
Shape of test_inpatient data : (9551, 30)
Shape of test_outpatient data : (125841, 27)


#### Cleaning: checking duplkicates,removing Nan values, filling missing values,dropping unwanted columns

In [5]:
train.head()

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Provider        5410 non-null   object
 1   PotentialFraud  5410 non-null   object
dtypes: object(2)
memory usage: 84.7+ KB


In [7]:
## Lets check whether  providers details are unique or not in train data

print(train.Provider.value_counts(sort=True,ascending=False).head(2))  # number of unique providers in train data.Check for duplicates

print('\n Total missing values in train :',train.isna().sum().sum())

print('\n Total missing values in train :',test.isna().sum().sum())

Provider
PRV51001    1
PRV55516    1
Name: count, dtype: int64

 Total missing values in train : 0

 Total missing values in train : 0


In [8]:
train_beneficiary.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200


In [9]:
train_beneficiary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   BeneID                           138556 non-null  object
 1   DOB                              138556 non-null  object
 2   DOD                              1421 non-null    object
 3   Gender                           138556 non-null  int64 
 4   Race                             138556 non-null  int64 
 5   RenalDiseaseIndicator            138556 non-null  object
 6   State                            138556 non-null  int64 
 7   County                           138556 non-null  int64 
 8   NoOfMonths_PartACov              138556 non-null  int64 
 9   NoOfMonths_PartBCov              138556 non-null  int64 
 10  ChronicCond_Alzheimer            138556 non-null  int64 
 11  ChronicCond_Heartfailure         138556 non-null  int64 
 12  ChronicCond_Kidn

In [10]:
train_beneficiary.isna().sum()

BeneID                                  0
DOB                                     0
DOD                                137135
Gender                                  0
Race                                    0
RenalDiseaseIndicator                   0
State                                   0
County                                  0
NoOfMonths_PartACov                     0
NoOfMonths_PartBCov                     0
ChronicCond_Alzheimer                   0
ChronicCond_Heartfailure                0
ChronicCond_KidneyDisease               0
ChronicCond_Cancer                      0
ChronicCond_ObstrPulmonary              0
ChronicCond_Depression                  0
ChronicCond_Diabetes                    0
ChronicCond_IschemicHeart               0
ChronicCond_Osteoporasis                0
ChronicCond_rheumatoidarthritis         0
ChronicCond_stroke                      0
IPAnnualReimbursementAmt                0
IPAnnualDeductibleAmt                   0
OPAnnualReimbursementAmt          

In [11]:
train_beneficiary=train_beneficiary.fillna(0)

In [12]:
train_beneficiary.isna().sum()

BeneID                             0
DOB                                0
DOD                                0
Gender                             0
Race                               0
RenalDiseaseIndicator              0
State                              0
County                             0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
ChronicCond_IschemicHeart          0
ChronicCond_Osteoporasis           0
ChronicCond_rheumatoidarthritis    0
ChronicCond_stroke                 0
IPAnnualReimbursementAmt           0
IPAnnualDeductibleAmt              0
OPAnnualReimbursementAmt           0
OPAnnualDeductibleAmt              0
dtype: int64

In [13]:
#Looking for duplicates in train_beneficiary

In [14]:
train_beneficiary.duplicated().sum()

0

In [15]:
train_inpatient.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [16]:
train_inpatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40474 entries, 0 to 40473
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BeneID                  40474 non-null  object 
 1   ClaimID                 40474 non-null  object 
 2   ClaimStartDt            40474 non-null  object 
 3   ClaimEndDt              40474 non-null  object 
 4   Provider                40474 non-null  object 
 5   InscClaimAmtReimbursed  40474 non-null  int64  
 6   AttendingPhysician      40362 non-null  object 
 7   OperatingPhysician      23830 non-null  object 
 8   OtherPhysician          4690 non-null   object 
 9   AdmissionDt             40474 non-null  object 
 10  ClmAdmitDiagnosisCode   40474 non-null  object 
 11  DeductibleAmtPaid       39575 non-null  float64
 12  DischargeDt             40474 non-null  object 
 13  DiagnosisGroupCode      40474 non-null  object 
 14  ClmDiagnosisCode_1      40474 non-null

In [17]:
train_inpatient.isna().sum()

BeneID                        0
ClaimID                       0
ClaimStartDt                  0
ClaimEndDt                    0
Provider                      0
InscClaimAmtReimbursed        0
AttendingPhysician          112
OperatingPhysician        16644
OtherPhysician            35784
AdmissionDt                   0
ClmAdmitDiagnosisCode         0
DeductibleAmtPaid           899
DischargeDt                   0
DiagnosisGroupCode            0
ClmDiagnosisCode_1            0
ClmDiagnosisCode_2          226
ClmDiagnosisCode_3          676
ClmDiagnosisCode_4         1534
ClmDiagnosisCode_5         2894
ClmDiagnosisCode_6         4838
ClmDiagnosisCode_7         7258
ClmDiagnosisCode_8         9942
ClmDiagnosisCode_9        13497
ClmDiagnosisCode_10       36547
ClmProcedureCode_1        17326
ClmProcedureCode_2        35020
ClmProcedureCode_3        39509
ClmProcedureCode_4        40358
ClmProcedureCode_5        40465
ClmProcedureCode_6        40474
dtype: int64

In [18]:
#dropping unwanted columns with nan values
train_inpatient=train_inpatient.drop(['ClmProcedureCode_6'],axis=1)

In [19]:
train_inpatient.shape

(40474, 29)

In [20]:
#filling the NaN values with 0

train_inpatient=train_inpatient.fillna(0)

In [21]:
train_inpatient.isna().sum()

BeneID                    0
ClaimID                   0
ClaimStartDt              0
ClaimEndDt                0
Provider                  0
InscClaimAmtReimbursed    0
AttendingPhysician        0
OperatingPhysician        0
OtherPhysician            0
AdmissionDt               0
ClmAdmitDiagnosisCode     0
DeductibleAmtPaid         0
DischargeDt               0
DiagnosisGroupCode        0
ClmDiagnosisCode_1        0
ClmDiagnosisCode_2        0
ClmDiagnosisCode_3        0
ClmDiagnosisCode_4        0
ClmDiagnosisCode_5        0
ClmDiagnosisCode_6        0
ClmDiagnosisCode_7        0
ClmDiagnosisCode_8        0
ClmDiagnosisCode_9        0
ClmDiagnosisCode_10       0
ClmProcedureCode_1        0
ClmProcedureCode_2        0
ClmProcedureCode_3        0
ClmProcedureCode_4        0
ClmProcedureCode_5        0
dtype: int64

In [22]:
train_inpatient.duplicated().sum()

0

In [23]:
train_outpatient.head()


,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [24]:
train_outpatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517737 entries, 0 to 517736
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  517737 non-null  object 
 1   ClaimID                 517737 non-null  object 
 2   ClaimStartDt            517737 non-null  object 
 3   ClaimEndDt              517737 non-null  object 
 4   Provider                517737 non-null  object 
 5   InscClaimAmtReimbursed  517737 non-null  int64  
 6   AttendingPhysician      516341 non-null  object 
 7   OperatingPhysician      90617 non-null   object 
 8   OtherPhysician          195046 non-null  object 
 9   ClmDiagnosisCode_1      507284 non-null  object 
 10  ClmDiagnosisCode_2      322357 non-null  object 
 11  ClmDiagnosisCode_3      203257 non-null  object 
 12  ClmDiagnosisCode_4      125596 non-null  object 
 13  ClmDiagnosisCode_5      74344 non-null   object 
 14  ClmDiagnosisCode_6  

In [25]:
#calculating the percentage of null values inn train_outpatient dataframe

missing = pd.concat([train_outpatient.isnull().sum(), 100 * train_outpatient.isnull().mean()], axis=1)
missing.columns=["count", "%"]
missing.sort_values(by="count",ascending=False)

,count,%
ClmProcedureCode_6,517737,100.000000
ClmProcedureCode_5,517737,100.000000
ClmProcedureCode_4,517735,99.999614
ClmProcedureCode_3,517733,99.999227
ClmProcedureCode_2,517701,99.993047
ClmProcedureCode_1,517575,99.968710
ClmDiagnosisCode_10,516654,99.790820
ClmDiagnosisCode_9,502899,97.134066
ClmDiagnosisCode_8,494825,95.574587
ClmDiagnosisCode_7,484776,93.633640


In [26]:
#dropping columns that have 99% and above null values
train_outpatient=train_outpatient.drop(['ClmProcedureCode_6','ClmProcedureCode_5','ClmProcedureCode_4','ClmProcedureCode_3','ClmProcedureCode_2','ClmProcedureCode_1'],axis=1)

In [27]:
#filling the rest with 0
train_outpatient=train_outpatient.fillna(0)

In [28]:
train_outpatient.isna().sum()

BeneID                    0
ClaimID                   0
ClaimStartDt              0
ClaimEndDt                0
Provider                  0
InscClaimAmtReimbursed    0
AttendingPhysician        0
OperatingPhysician        0
OtherPhysician            0
ClmDiagnosisCode_1        0
ClmDiagnosisCode_2        0
ClmDiagnosisCode_3        0
ClmDiagnosisCode_4        0
ClmDiagnosisCode_5        0
ClmDiagnosisCode_6        0
ClmDiagnosisCode_7        0
ClmDiagnosisCode_8        0
ClmDiagnosisCode_9        0
ClmDiagnosisCode_10       0
DeductibleAmtPaid         0
ClmAdmitDiagnosisCode     0
dtype: int64

In [29]:
train_outpatient.duplicated().sum()

0

In [30]:
test.head()

,Provider
0,PRV51002
1,PRV51006
2,PRV51009
3,PRV51010
4,PRV51018


In [31]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1353 entries, 0 to 1352
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Provider  1353 non-null   object
dtypes: object(1)
memory usage: 10.7+ KB


In [32]:
test.duplicated().sum()

0

In [33]:
test_beneficiary.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11007,1940-09-01,2009-12-01,1,2,0,45,610,12,12,...,2,1,2,1,1,2,0,0,1490,160
2,BENE11010,1936-07-01,NaN,2,1,0,41,30,12,12,...,2,1,1,1,2,2,0,0,1170,660
3,BENE11011,1914-03-01,NaN,2,2,0,1,360,12,12,...,1,1,2,2,1,1,5000,1068,250,320
4,BENE11014,1938-04-01,NaN,2,1,Y,45,780,12,12,...,1,2,1,2,2,2,21260,2136,120,100


In [34]:
test_beneficiary.isna().sum()

BeneID                                 0
DOB                                    0
DOD                                63394
Gender                                 0
Race                                   0
RenalDiseaseIndicator                  0
State                                  0
County                                 0
NoOfMonths_PartACov                    0
NoOfMonths_PartBCov                    0
ChronicCond_Alzheimer                  0
ChronicCond_Heartfailure               0
ChronicCond_KidneyDisease              0
ChronicCond_Cancer                     0
ChronicCond_ObstrPulmonary             0
ChronicCond_Depression                 0
ChronicCond_Diabetes                   0
ChronicCond_IschemicHeart              0
ChronicCond_Osteoporasis               0
ChronicCond_rheumatoidarthritis        0
ChronicCond_stroke                     0
IPAnnualReimbursementAmt               0
IPAnnualDeductibleAmt                  0
OPAnnualReimbursementAmt               0
OPAnnualDeductib

In [35]:
#calculating the percentage of null values in each column
missing = pd.concat([test_beneficiary.isnull().sum(), 100 * test_beneficiary.isnull().mean()], axis=1)
missing.columns=["count", "%"]
missing.sort_values(by="count",ascending=False)

,count,%
DOD,63394,99.102676
BeneID,0,0.000000
ChronicCond_Cancer,0,0.000000
OPAnnualReimbursementAmt,0,0.000000
IPAnnualDeductibleAmt,0,0.000000
IPAnnualReimbursementAmt,0,0.000000
ChronicCond_stroke,0,0.000000
ChronicCond_rheumatoidarthritis,0,0.000000
ChronicCond_Osteoporasis,0,0.000000
ChronicCond_IschemicHeart,0,0.000000


In [36]:
test_inpatient.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,NaN,2009-09-09,...,5990,570,41071,4280,4443.0,5849.0,NaN,NaN,NaN,NaN
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,NaN,2008-12-25,...,4549,29570,34831,NaN,5551.0,NaN,NaN,NaN,NaN,NaN
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,NaN,NaN,2009-12-09,...,78650,7813,4254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,NaN,2009-06-23,...,4254,41400,5849,41401,8152.0,3320.0,NaN,NaN,NaN,NaN
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,NaN,NaN,2009-01-20,...,V6109,7242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
test_inpatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BeneID                  9551 non-null   object 
 1   ClaimID                 9551 non-null   object 
 2   ClaimStartDt            9551 non-null   object 
 3   ClaimEndDt              9551 non-null   object 
 4   Provider                9551 non-null   object 
 5   InscClaimAmtReimbursed  9551 non-null   int64  
 6   AttendingPhysician      9520 non-null   object 
 7   OperatingPhysician      5589 non-null   object 
 8   OtherPhysician          1013 non-null   object 
 9   AdmissionDt             9551 non-null   object 
 10  ClmAdmitDiagnosisCode   9551 non-null   object 
 11  DeductibleAmtPaid       9355 non-null   float64
 12  DischargeDt             9551 non-null   object 
 13  DiagnosisGroupCode      9551 non-null   object 
 14  ClmDiagnosisCode_1      9551 non-null   

In [38]:
missing = pd.concat([test_inpatient.isnull().sum(), 100 * test_inpatient.isnull().mean()], axis=1)
missing.columns=["count", "%"]
missing.sort_values(by="count",ascending=False)

,count,%
ClmProcedureCode_6,9551,100.000000
ClmProcedureCode_5,9549,99.979060
ClmProcedureCode_4,9522,99.696367
ClmProcedureCode_3,9328,97.665166
ClmDiagnosisCode_10,8664,90.713014
OtherPhysician,8538,89.393781
ClmProcedureCode_2,8297,86.870485
ClmProcedureCode_1,4118,43.115904
OperatingPhysician,3962,41.482567
ClmDiagnosisCode_9,3238,33.902209


In [39]:
#dropping unwanted columns
test_inpatient=test_inpatient.drop(['ClmProcedureCode_3','ClmProcedureCode_4','ClmProcedureCode_5','ClmProcedureCode_6'],axis=1)

In [40]:
test_inpatient=test_inpatient.fillna(0)

In [41]:
test_inpatient.duplicated().sum()

0

In [42]:
test_outpatient.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11001,CLM392397,2009-06-02,2009-06-02,PRV55962,30,PHY347633,NaN,PHY347633,V5832,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,BENE11001,CLM430760,2009-06-23,2009-06-23,PRV56112,30,PHY381777,NaN,PHY381777,9594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,BENE11007,CLM233081,2009-03-07,2009-03-07,PRV56979,200,PHY425311,NaN,PHY425311,7248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11007,CLM496381,2009-07-29,2009-07-29,PRV56573,10,PHY393253,PHY347995,NaN,58889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5939
4,BENE11007,CLM521391,2009-08-12,2009-08-12,PRV56573,10,PHY417685,NaN,PHY382041,V666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [43]:
test_outpatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125841 entries, 0 to 125840
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  125841 non-null  object 
 1   ClaimID                 125841 non-null  object 
 2   ClaimStartDt            125841 non-null  object 
 3   ClaimEndDt              125841 non-null  object 
 4   Provider                125841 non-null  object 
 5   InscClaimAmtReimbursed  125841 non-null  int64  
 6   AttendingPhysician      125525 non-null  object 
 7   OperatingPhysician      21604 non-null   object 
 8   OtherPhysician          47619 non-null   object 
 9   ClmDiagnosisCode_1      123263 non-null  object 
 10  ClmDiagnosisCode_2      78110 non-null   object 
 11  ClmDiagnosisCode_3      49266 non-null   object 
 12  ClmDiagnosisCode_4      30470 non-null   object 
 13  ClmDiagnosisCode_5      17966 non-null   object 
 14  ClmDiagnosisCode_6  

In [44]:
missing = pd.concat([test_outpatient.isnull().sum(), 100 * test_outpatient.isnull().mean()], axis=1)
missing.columns=["count", "%"]
missing.sort_values(by="count",ascending=False)

,count,%
ClmProcedureCode_6,125841,100.000000
ClmProcedureCode_5,125841,100.000000
ClmProcedureCode_4,125841,100.000000
ClmProcedureCode_3,125839,99.998411
ClmProcedureCode_2,125832,99.992848
ClmProcedureCode_1,125807,99.972982
ClmDiagnosisCode_10,125578,99.791006
ClmDiagnosisCode_9,122278,97.168649
ClmDiagnosisCode_8,120310,95.604771
ClmDiagnosisCode_7,117871,93.666611


In [45]:
test_outpatient=test_outpatient.drop(['ClmProcedureCode_6','ClmProcedureCode_5','ClmProcedureCode_4','ClmProcedureCode_3','ClmProcedureCode_2','ClmProcedureCode_1'],axis=1)

In [46]:
test_outpatient=test_outpatient.fillna(0)

In [47]:
test_outpatient.duplicated().sum()

0

In [48]:
test_outpatient.shape

(125841, 21)

In [49]:
# cheking unique values in categorigal featues 
train_bene_col=train_beneficiary.columns
train_bene_col=list(train_bene_col)
train_bene_col=train_bene_col[3:-4] # removing DOB,DOD and other continues feateus.
for i in train_bene_col:
    print(i,"=",train_beneficiary[i].unique())

Gender = [1 2]
Race = [1 2 3 5]
RenalDiseaseIndicator = ['0' 'Y']
State = [39 52 24 23 45 15 44 41  1 14  7 13 34 31  5 49 46  6 38 10 26  3 20 33
 18 27 51 29 42 36 11 12 30 19  4 25 17 16 32  2 21 50 22 54 28 43 35 37
 47  8  9 53]
County = [230 280 590 270 680 810 610 140  30 360 982  10 780 400 120  50 470  20
 892 200 170 500 710 760  90 130 141 150 700 390 620 260 930 734 250 510
  60 563 570  40 340 100 290 660 350 770 180 460 580 430 880 690 310 550
 190 490 600 220 790 990 370 420   0 980 480 450 960 160 800 410 520 380
 991 110 540 910 240 440  70 860 640 870 560 650 830 330 820  25 451 320
 530 831 950  80 921 940 630 720 989 331 840 758 421 722 730 561 750 791
 947 551 801 953 411 681 300 970 757 241 210 988 890 756 221 843 581 702
 999  11 711 670 328 943 981 591 754 986 431 194 885 740 900 871 641 653
 911 879 992 622 291 812 562 976 920 311 288 752 811 850 191 461 821 223
 792 984 611 882 794 891 962 983 772 842 341 841 211 511 971 881 564 541
 948 913 941 391  55 888 63

### Data preprocessing

1) only data of death column has Nan values
2) we have to change gender to 1 and 0 from 1 and 2.(for easy interpritaion)
3) In RenalDiseaseIndicator we have to replace Y with 1.
4) we have to make weather a beneficiary has chronic condition or not by replacing 2 with 0 (no cronic condition).From this we can add another feature called Total_chronic_condition by simply adding them.

In [50]:
# replacing 2 with 0 in gender, train_beneficary[gender]
train_beneficiary['Gender']=train_beneficiary['Gender'].replace(2,0)
train_beneficiary['Gender'].head()
# replacing in test beneficary data
test_beneficiary['Gender']=test_beneficiary['Gender'].replace(2,0)
test_beneficiary['Gender'].head()


0    1
1    1
2    0
3    0
4    0
Name: Gender, dtype: int64

In [51]:
# replacing Y with 1 in RenalDiseaseIndicator
train_beneficiary['RenalDiseaseIndicator']=train_beneficiary['RenalDiseaseIndicator'].replace('Y',1)
# repacling in test data
test_beneficiary['RenalDiseaseIndicator']=test_beneficiary['RenalDiseaseIndicator'].replace('Y',1)
test_beneficiary['RenalDiseaseIndicator'].head()

0    0
1    0
2    0
3    0
4    1
Name: RenalDiseaseIndicator, dtype: object

In [52]:
# replacing 2, with 0 in all 10 chronic condition, here we considering zero as no chornic condition
train_beneficiary['ChronicCond_Alzheimer']=train_beneficiary['ChronicCond_Alzheimer'].replace(2,0)
train_beneficiary['ChronicCond_Cancer']=train_beneficiary['ChronicCond_Cancer'].replace(2,0)
train_beneficiary['ChronicCond_Depression']=train_beneficiary['ChronicCond_Depression'].replace(2,0)
train_beneficiary['ChronicCond_Diabetes']=train_beneficiary['ChronicCond_Diabetes'].replace(2,0)
train_beneficiary['ChronicCond_Heartfailure']=train_beneficiary['ChronicCond_Heartfailure'].replace(2,0)
train_beneficiary['ChronicCond_IschemicHeart']=train_beneficiary['ChronicCond_IschemicHeart'].replace(2,0)
train_beneficiary['ChronicCond_KidneyDisease']=train_beneficiary['ChronicCond_KidneyDisease'].replace(2,0)
train_beneficiary['ChronicCond_ObstrPulmonary']=train_beneficiary['ChronicCond_ObstrPulmonary'].replace(2,0)
train_beneficiary['ChronicCond_Osteoporasis']=train_beneficiary['ChronicCond_Osteoporasis'].replace(2,0)
train_beneficiary['ChronicCond_rheumatoidarthritis']=train_beneficiary['ChronicCond_rheumatoidarthritis'].replace(2,0)
train_beneficiary['ChronicCond_stroke']=train_beneficiary['ChronicCond_stroke'].replace(2,0)

# replacing in test data
test_beneficiary['ChronicCond_Alzheimer']=test_beneficiary['ChronicCond_Alzheimer'].replace(2,0)
test_beneficiary['ChronicCond_Cancer']=test_beneficiary['ChronicCond_Cancer'].replace(2,0)
test_beneficiary['ChronicCond_Depression']=test_beneficiary['ChronicCond_Depression'].replace(2,0)
test_beneficiary['ChronicCond_Diabetes']=test_beneficiary['ChronicCond_Diabetes'].replace(2,0)
test_beneficiary['ChronicCond_Heartfailure']=test_beneficiary['ChronicCond_Heartfailure'].replace(2,0)
test_beneficiary['ChronicCond_IschemicHeart']=test_beneficiary['ChronicCond_IschemicHeart'].replace(2,0)
test_beneficiary['ChronicCond_KidneyDisease']=test_beneficiary['ChronicCond_KidneyDisease'].replace(2,0)
test_beneficiary['ChronicCond_ObstrPulmonary']=test_beneficiary['ChronicCond_ObstrPulmonary'].replace(2,0)
test_beneficiary['ChronicCond_Osteoporasis']=test_beneficiary['ChronicCond_Osteoporasis'].replace(2,0)
test_beneficiary['ChronicCond_rheumatoidarthritis']=test_beneficiary['ChronicCond_rheumatoidarthritis'].replace(2,0)
test_beneficiary['ChronicCond_stroke']=test_beneficiary['ChronicCond_stroke'].replace(2,0)


In [53]:
train_beneficiary['ChronicCond_Alzheimer'].unique()

array([1, 0])

### Beneficiary data

In [54]:
train_beneficiary.columns

Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt'],
      dtype='object')

#### Dead or Alive

In [55]:
train_beneficiary.loc[train_beneficiary.DOD.isna(),'Dead_or_Alive']=0
train_beneficiary.loc[train_beneficiary.DOD.notna(),'Dead_or_Alive']=1
# test data

test_beneficiary.loc[test_beneficiary.DOD.isna(),"Dead_or_Alive"]=0
test_beneficiary.loc[test_beneficiary.DOD.notna(),"Dead_or_Alive"]=1

#### Age

we will calculate age from the latest date present in our data

In [56]:

train_beneficiary = train_beneficiary[train_beneficiary['DOD'] != 0]
train_beneficiary.DOD.unique()

array(['2009-12-01', '2009-09-01', '2009-05-01', '2009-07-01',
       '2009-08-01', '2009-10-01', '2009-06-01', '2009-04-01',
       '2009-02-01', '2009-11-01', '2009-03-01'], dtype=object)

In [57]:

train_beneficiary['DOB']=pd.to_datetime(train_beneficiary['DOB'],format='%Y-%m-%d')
train_beneficiary['DOD']=pd.to_datetime(train_beneficiary['DOD'],format='%Y-%m-%d')
# test
test_beneficiary['DOD']=pd.to_datetime(test_beneficiary['DOD'],format='%Y-%m-%d')
test_beneficiary['DOB']=pd.to_datetime(test_beneficiary['DOB'],format='%Y-%m-%d')

# subracting dod form dob to get the age according to it.
train_beneficiary['Age']= round((train_beneficiary['DOD']-train_beneficiary['DOB']).dt.days/365)
#testB
test_beneficiary['Age']=round((test_beneficiary['DOD']-test_beneficiary['DOB']).dt.days/365)

In [58]:
train_beneficiary.Age.unique()
test_beneficiary.Age.unique()

array([ nan,  69.,  70.,  81.,  71.,  99.,  68.,  84.,  59.,  80.,  94.,
        67.,  74.,  66.,  73.,  86.,  47.,  54.,  90.,  78.,  82.,  77.,
        83.,  52.,  76.,  72.,  53., 100.,  75.,  62.,  92.,  50.,  56.,
        79.,  49.,  85.,  87.,  89.,  61.,  55.,  91.,  88.,  44.,  57.,
        65.,  63.,  42.,  60.,  64.,  95.,  51.,  93.,  96.,  58.,  43.,
        46.,  39.,  48.,  37.,  98.])

In [59]:
# fill the nan vlaues accorindig to the latest date presnt in DoD.
train_beneficiary['Age']=train_beneficiary['Age'].fillna(round((pd.to_datetime('2009-12-01',format='%Y-%m-%d')-train_beneficiary['DOB']).dt.days/365))

# test
test_beneficiary['Age']=test_beneficiary['Age'].fillna(round((pd.to_datetime('2009-12-01',format='%Y-%m-%d')-test_beneficiary['DOB']).dt.days/365))
# cheking
train_beneficiary['Age'].isna().any()
test_beneficiary['Age'].isna().any()

False

In [60]:
age_count=train_beneficiary.Age.value_counts()

### INPatient data

In [61]:
train_inpatient.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5'],
      dtype='object')

#### Admitted_or_Not

Adding a new column for weather a patient is admitted or not for inpaitent data we will indicate it with 1.



In [62]:
#Traindata
train_inpatient['Admitted_or_Not']=1
# test_data
test_inpatient['Admitted_or_Not']=1

### Outpatient

#### Admitted_or_Not

Adding a new column for weather a patient is admitted or not for outpatient data we will indicate it with 0.

In [63]:
#Traindata
train_outpatient['Admitted_or_Not']=0
# test_data
test_outpatient['Admitted_or_Not']=0


### Merging data

1) first we merge inpatient data and outpatient by common columns present in both the data frame.
2) second we merge the merged data set in first step with beneficiary data by BeneId (Beneficiary id).
3) Third we will merge the train data with above merged data set by Provider id.

### Merging inpatient and outpatient data

In [64]:
print(train_inpatient.shape,train_outpatient.shape)

(40474, 30) (517737, 22)


In [65]:
comm_col=[]
for i in train_inpatient.columns:
    if i in train_outpatient.columns:
        comm_col.append(i)
        print(i)
len(comm_col)

BeneID
ClaimID
ClaimStartDt
ClaimEndDt
Provider
InscClaimAmtReimbursed
AttendingPhysician
OperatingPhysician
OtherPhysician
ClmAdmitDiagnosisCode
DeductibleAmtPaid
ClmDiagnosisCode_1
ClmDiagnosisCode_2
ClmDiagnosisCode_3
ClmDiagnosisCode_4
ClmDiagnosisCode_5
ClmDiagnosisCode_6
ClmDiagnosisCode_7
ClmDiagnosisCode_8
ClmDiagnosisCode_9
ClmDiagnosisCode_10
Admitted_or_Not


22

In [66]:
IN_OUT_train=pd.merge(train_inpatient,train_outpatient,left_on=comm_col,right_on=comm_col,how='outer')
IN_OUT_test=pd.merge(test_inpatient,test_outpatient,left_on=comm_col,right_on=comm_col,how='outer')
IN_OUT_train.columns
IN_OUT_test.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'Admitted_or_Not'],
      dtype='object')

#### Admitted_days

we will get admitted_days by subtraching DiscargeDt to AdmissinonDt.



In [67]:
IN_OUT_train['AdmissionDt'].isna().any()

True

In [68]:
IN_OUT_train['AdmissionDt']=pd.to_datetime(IN_OUT_train['AdmissionDt'],format='%Y-%m-%d')
IN_OUT_train['DischargeDt']=pd.to_datetime(IN_OUT_train['DischargeDt'],format='%Y-%m-%d')
#test
IN_OUT_test['AdmissionDt']=pd.to_datetime(IN_OUT_test['AdmissionDt'],format='%Y-%m-%d')
IN_OUT_test['DischargeDt']=pd.to_datetime(IN_OUT_test['DischargeDt'],format='%Y-%m-%d')

IN_OUT_train['Admitted_days']=round((IN_OUT_train['DischargeDt']-IN_OUT_train['AdmissionDt']).dt.days)
#test
IN_OUT_test['Admitted_days']=round((IN_OUT_test['DischargeDt']-IN_OUT_test['AdmissionDt']).dt.days)

In [69]:
#filling nan values with 1 because patient is admitted for mininmun day is 1.
IN_OUT_train['Admitted_days']=IN_OUT_train['Admitted_days'].fillna(1)
IN_OUT_test['Admitted_days']=IN_OUT_test['Admitted_days'].fillna(1)
IN_OUT_train['Admitted_days'].isna().any()

False

#### Claim_time

we will calculate claim_time by subtracting claim_start_date by claim_end_date.

In [70]:
IN_OUT_train['ClaimStartDt'].isna().any()
IN_OUT_train['ClaimEndDt'].isna().any()

False

In [71]:
IN_OUT_train['ClaimStartDt']=pd.to_datetime(IN_OUT_train['ClaimStartDt'],format='%Y-%m-%d')
IN_OUT_train['ClaimEndDt']=pd.to_datetime(IN_OUT_train['ClaimEndDt'],format='%Y-%m-%d')
#test
IN_OUT_test['ClaimStartDt']=pd.to_datetime(IN_OUT_test['ClaimStartDt'],format='%Y-%m-%d')
IN_OUT_test['ClaimEndDt']=pd.to_datetime(IN_OUT_test['ClaimEndDt'],format='%Y-%m-%d')

IN_OUT_train['Claim_time']=round((IN_OUT_train['ClaimEndDt']-IN_OUT_train['ClaimStartDt']).dt.days)+1 # adding one becase atleast 1 day to process the claim
#test
IN_OUT_test['Claim_time']=round((IN_OUT_test['ClaimEndDt']-IN_OUT_test['ClaimStartDt']).dt.days)+1 

#### Amount_get

we will calulate amount patient will get.

In [72]:
IN_OUT_train['InscClaimAmtReimbursed'].isna().any()
IN_OUT_train['DeductibleAmtPaid'].isna().any()

False

In [73]:
IN_OUT_train['DeductibleAmtPaid']=IN_OUT_train['DeductibleAmtPaid'].fillna(0)
#test
IN_OUT_test['DeductibleAmtPaid']=IN_OUT_test['DeductibleAmtPaid'].fillna(0)

In [74]:
IN_OUT_train['Amount_get']=IN_OUT_train['InscClaimAmtReimbursed']-IN_OUT_train['DeductibleAmtPaid']
#test
IN_OUT_test['Amount_get']=IN_OUT_test['InscClaimAmtReimbursed']-IN_OUT_test['DeductibleAmtPaid']


In [75]:
IN_OUT_train=IN_OUT_train.fillna(0)

In [76]:
print(IN_OUT_train.shape)
IN_OUT_test.shape

(558211, 33)


(135392, 30)

### Merging merged inpatient and outpatient data with beneficiary data

In [77]:
in_out_bene_train=pd.merge(IN_OUT_train,train_beneficiary,on='BeneID',how='inner')
#test data
in_out_bene_test=pd.merge(IN_OUT_test,test_beneficiary,on="BeneID",how='inner')

In [78]:
in_out_bene_train.head()



,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Dead_or_Alive,Age
0,BENE11231,CLM42956,2009-03-17,2009-03-18,PRV56703,4000,PHY419888,0,0,2009-03-17 00:00:00,...,1,1,0,0,4000,1068,300,30,1.0,71.0
1,BENE13189,CLM39798,2009-02-24,2009-03-09,PRV52021,45000,PHY429938,PHY429938,0,2009-02-24 00:00:00,...,1,0,1,0,45780,1068,11790,6030,1.0,70.0
2,BENE13189,CLM110109,2008-12-13,2009-01-02,PRV53761,1400,PHY411435,0,0,0,...,1,0,1,0,45780,1068,11790,6030,1.0,70.0
3,BENE13189,CLM138192,2009-01-15,2009-02-04,PRV53799,400,PHY431281,0,0,0,...,1,0,1,0,45780,1068,11790,6030,1.0,70.0
4,BENE13189,CLM147321,2009-01-20,2009-02-09,PRV51949,90,PHY421150,PHY320133,0,0,...,1,0,1,0,45780,1068,11790,6030,1.0,70.0


In [79]:
print(in_out_bene_train.shape,in_out_bene_test.shape)

(4131, 59) (135392, 56)


### Merging in_out_bene data with train_labels data


In [80]:
final_data_train=pd.merge(in_out_bene_train,train,on='Provider',how='inner')
final_data_test=pd.merge(in_out_bene_test,test,on='Provider',how='inner')
print(final_data_train.shape,final_data_test.shape)

(4131, 60) (135392, 56)


In [81]:
final_data_test.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'Admitted_or_Not', 'Admitted_days', 'Claim_time', 'Amount_get', 'DOB',
       'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator', 'State', 'County',
       'NoOfMonths_PartACov', 'NoOfMonths_PartBCov', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCon